In [1]:
import os
import re

import numpy as np

In [3]:
def get_adjacents(data_padded, c, r_start, r_end):
    '''
    line[r_start:r_end] is the number. 
    '''
    # number
    data_padded[c+1, r_start+1:r_end+1]
    adjacents = []
    # above
    adjacents.extend(list(data_padded[c, r_start:r_end+2]))
    # right and left
    adjacents.append(data_padded[c+1, r_start])
    adjacents.append(data_padded[c+1, r_end+1])
    # below
    adjacents.extend(data_padded[c+2, r_start:r_end+2])
    
    return adjacents

In [5]:
is_test = False

if is_test:
    test_txt = 'input_.txt'
else:
    test_txt = 'input.txt'
    
with open(test_txt) as f:
    lines = f.read().strip().split('\n')

data = np.array([np.array(list(line)) for line in lines])
data_padded = np.pad(data, 1, mode='constant', constant_values=('.'))
#data_padded

In [13]:
part_numbers_all = 0
for c, line in enumerate(lines):
    numbers = re.findall('\d+', line)
    if len(numbers) > 0: # if there are numbers in the line.
        for number in numbers:
            # row where the number start/end
            r_start = line.find(number)
            r_end = r_start + len(number)
            
            ## get adjacent cells.
            adjacents = get_adjacents(data_padded, c, r_start, r_end)
            adjacents_ = [x for x in adjacents if x not in ['.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']]
            if len(adjacents_) > 0:
                print(f"{number} ({c}, {r_start}-{r_end}): {adjacents_}")
                print(f"{adjacents}")
                part_numbers_all += int(number)
print(f"the sum of all of the part numbers in the engine schematic: {part_numbers_all}")

3 (0, 25-26): ['*']
['.', '.', '.', '.', '.', '*', '.', '.']
94 (0, 65-67): ['*']
['.', '.', '.', '.', '.', '.', '*', '.', '.', '.']
806 (0, 82-85): ['*']
['.', '.', '.', '.', '.', '.', '.', '.', '*', '.', '.', '.']
793 (0, 117-120): ['=']
['.', '.', '.', '.', '.', '.', '.', '=', '.', '.', '.', '.']
186 (0, 131-134): ['*']
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '*', '.']
574 (1, 15-18): ['*']
['.', '.', '.', '.', '.', '*', '.', '.', '.', '.', '.', '.']
587 (1, 19-22): ['*']
['.', '.', '.', '.', '.', '.', '.', '.', '*', '.', '.', '.']
161 (1, 33-36): ['*']
['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '*']
904 (1, 42-45): ['&']
['.', '.', '.', '.', '.', '.', '.', '.', '&', '.', '.', '.']
412 (1, 52-55): ['*']
['.', '.', '.', '.', '.', '.', '.', '*', '.', '.', '.', '.']
637 (1, 122-125): ['*']
['.', '.', '.', '.', '.', '.', '.', '*', '.', '.', '.', '.']
614 (2, 2-5): ['/']
['.', '.', '/', '.', '.', '.', '.', '.', '.', '.', '.', '.']
831 (2, 7-10): ['*']
['.', '.', '